### NICO

Implementation based on the matlab code by M. Rabbat

In [1]:
import numpy as np
from random import shuffle, random
from scipy.special import comb
from scipy.sparse import csr_matrix
import itertools

Initialize parameters

In [2]:
n = 2 #number of nodes in the network
T = 100 #number of paths
Nm = 4 #number of nodes per path
# np.random.seed(2)
np.random.seed(1337)

In [3]:
A = np.random.rand(n, n)
A = A / A.sum(axis = 1, keepdims=True)

In [4]:
pi = np.random.rand(n, 1)
pi = pi / pi.sum(axis=0, keepdims=True)

Generate some paths according to this Markov model

In [5]:
X = np.zeros((T, Nm))

In [6]:
# Generate random numbers for testing purposes. Change to random() when it is ready
R_out = np.random.rand(T,1)
R_in = np.random.rand(T, Nm)

In [7]:
cumprobs = pi.cumsum(axis = 0)

In [8]:
iterator = 0
for walk in X:
#     Sample the starting node from Pi
#     larger = (cumprobs >= random()).nonzero()
    larger = (cumprobs >= R_out[iterator][0]).nonzero()
    walk[0] = larger[0][0]
#     Sample remaining nodes in the path by taking a random walk
    for i in range(1, Nm):
        cumprobs_in = A[int(walk[i - 1]),:].cumsum(axis=0)
#       larger = (cumprobs >= random()).nonzero()
        larger = (cumprobs_in >= R_in[iterator][i]).nonzero()
        walk[i] = larger[0][0]
    iterator += 1

Shuffle observations

In [9]:
# Y = X.copy()
# for walk in Y:
#     shuffle(walk)

In [10]:
# numTrials = 50

Utils

In [11]:
def normalize_csr_rows(csr_mat):
    row_sums = np.array(csr_mat.sum(axis=1))[:,0]
    row_indices, col_indices = csr_mat.nonzero()
    csr_mat.data /= row_sums[col_indices]

NICO implementation

In [12]:
def nico(X, n):
#     T = np.shape(X)[0]
    
    #number of nodes in each path
#     size = lambda array: len(array)
#     Nm = np.apply_along_axis(size, 1, X)
    
    #Init pi_hat
    #Assume all states appear at least once in the data
    pi_hat = 1 + 0.3 * np.random.rand(n, 1)
    pi_hat = pi_hat / pi_hat.sum(axis = 0, keepdims = True)
    
    # Construct A_hat as a sparse matrix
    # First determine an upperbound on the number of non-zero entries
    ii = []
    jj = []

    for walk in X:
        V = np.array(list(itertools.combinations(walk, 2)))
        ii.append(list(V[:, 0]))
        jj.append(list(V[:, 1]))

    ii = [item for sublist in ii for item in sublist]
    jj = [item for sublist in jj for item in sublist]
    ss = np.ones(len(ii))
    
    A_hat = csr_matrix((ss, (ii, jj)), shape = (n,n))
    A_hat = (A_hat + A_hat.transpose()) / 2
    A_hat_copy = A_hat.copy()
    A_hat_copy.data.fill(1)
    
    A_hat = A_hat_copy + 0.4 * csr_matrix((np.random.random((A_hat.nnz)),A_hat.nonzero()), shape=A_hat.shape)
    A_hat = A_hat.transpose()
    
    #Normalize A_hat
    normalize_csr_rows(A_hat)
#     row_sums = np.array(A_hat.sum(axis=1))[:,0]
#     row_indices, col_indices = A_hat.nonzero()
#     A_hat.data /= row_sums[row_indices]
    
    return A_hat, pi_hat

In [13]:
A_hat, pi_hat = nico(X, n)

In [14]:
A_hat.toarray()

array([[0.16808399, 0.17387978, 0.17518154, 0.14682224, 0.17895884,
        0.1570736 ],
       [0.16136484, 0.16781939, 0.17191685, 0.18090937, 0.17451246,
        0.1434771 ],
       [0.17132931, 0.16467022, 0.15711988, 0.17866546, 0.14455837,
        0.18365676],
       [0.14893897, 0.17077987, 0.15725988, 0.18613615, 0.17194182,
        0.16494331],
       [0.17186718, 0.15980495, 0.16509716, 0.14996267, 0.19982792,
        0.15344011],
       [0.1649417 , 0.15230273, 0.1652519 , 0.19920753, 0.15139216,
        0.16690397]])

In [15]:
pi_hat

array([[0.15786837],
       [0.17147875],
       [0.16721507],
       [0.17649442],
       [0.17787994],
       [0.14906346]])